# Optimizacion Industrial con Computacion Evolutiva
<b> Diplomatura de Especialización en Desarrollo de Aplicaciones con Inteligencia Artificial</b>

* Antony Marin, 
* Oscar Serquén
* Christian Solano 

## DESAFIO: Algoritmo Genetico para resolver el tablero de Sudoku

Sudoku es un juego de lógica japonés en donde el objetivo es rellenar una celda de 9×9 casillas con números del 1 al 9, de manera que cada columna, fila y cada uno de las subceldas 3×3 que componen la celda 9×9 contengan cada número solo una vez.

Este trabajo pretende solucionar tableros de Sudoku un juego ampliamente conocido, con el fin de poner en práctica los conceptos de un algoritmo genético tales como: población, individuo, cromosoma, fitness, selección, cruzamiento, mutación, generación entre otros.


In [ ]:
import sys
import time
#import random
from random import shuffle, random, sample, randint , uniform , randrange
import numpy as np
from copy import deepcopy
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import heapq
import math

In [ ]:
class Individual(object):   
   
    def __init__(self, chromosome):
            self.chromosome = chromosome[:]
            self.fitness = -1  # -1 indica que el individuo no ha sido evaluado
     
    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
        c = randrange(len(self.chromosome))
        ind1 = Individual(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]   
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other"
        for i in range(len(self.chromosome)):
            if uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual(chromosome1)
        ind2 = Individual(chromosome2)
        return [ind1, ind2]     
    
    def crossover_orderpermutation(self, other):
        "Uniform order-based crossover. Genera 2 cromosomas hijos  que tambien deben ser permutaciones" 
        "Retorna dos nuevos individuos del cruzamiento de permutacion entre self y other" 
        "Escoge aleatoriamente la mitad de posiciones del cromosoma self y lo copia a un hijo ind1 en las mismas posiciones" 
        "Los numeros faltantes los copia de other en el mismo orden en que aparecen en other "
        "El hijo ind2 se crea de la misma forma intercambiando self por other"
        
        
        pos_constantes = []
        pos_constantes = sample(range(len(self.chromosome)),len(self.chromosome)//2)
        cromosomo1 = [-1]*len(self.chromosome) 
        cromosomo2 = [-1]*len(self.chromosome) 
        
        for index in pos_constantes:
            cromosomo1[index]= self.chromosome[index]
            cromosomo2[index] = other.chromosome[index]
            
        
        for i in range(len(self.chromosome)):
            if(cromosomo1[i] == -1):
                for j in range(len(other.chromosome)):
                    if other.chromosome[j] not in cromosomo1:
                        cromosomo1[i] = other.chromosome[j]
                        break
                        
                for j in range(len(self.chromosome)):
                    if self.chromosome[j] not in cromosomo2:
                        cromosomo2[i] = self.chromosome[j]
                        break          
        
        ind1 = Individual(cromosomo1)
        ind2 = Individual(cromosomo2)
        return [ind1, ind2]          
    
    
    def mutate_position(self): 
        """       mutate_position
        Cambia aleatoriamente un alelo de un gen."""
        mutated_chromosome = deepcopy(self.chromosome)
        mutGene = randrange(1,len(mutated_chromosome)) 
        newAllele = randrange(1,9)
        mutated_chromosome[mutGene] = newAllele
        return Individual(mutated_chromosome)    
        
    def mutate_swap(self):
        "Escoge dos genes e intercambia sus alelos"
        mutated_chromosome = deepcopy(self.chromosome[:])
        mutGen1 = randrange(1,len(mutated_chromosome))
        mutGen2 = randrange(1,len(mutated_chromosome))
        temp = mutated_chromosome[mutGen1]
        mutated_chromosome[mutGen1] = mutated_chromosome[mutGen2]
        mutated_chromosome[mutGen2] = temp
        return Individual(mutated_chromosome)

    def mutate_make_neighborBoard(self):
        " Genera un tablero nuevo. Se escoje aleatoriamente un bloque y se intercambia dos valores (no dados en el tablero original)"
        mutated_chromosome = deepcopy(self.chromosome)
        initialEntries = np.arange(81)[np.array(mutated_chromosome) > 0]
        block = randint(0,8)  # escoje un bloque aleatoriamente
        num_in_block = len(get_block_indices(block,initialEntries,ignore_originals=True)) #cantidad de posiciones que se puede alterar en el bloque
        if  num_in_block>1 :
          random_squares = sample(range(num_in_block),2) # escoje dos posiciones aleatorias del bloque para intercambiar valores
          square1, square2 = [get_block_indices(block,initialEntries,ignore_originals=True)[ind] for ind in random_squares]
          mutated_chromosome[square1], mutated_chromosome[square2] = mutated_chromosome[square2], mutated_chromosome[square1] # intercambia los valores de las posiciones
        else:mutated_chromosome
        return Individual(mutated_chromosome)  

### Funciones para inicializar y visualizar Poblaciones

In [ ]:
def readBoardFromFile(filename):
    fd = open(filename,"r+")    
    puzzle = eval(fd.readline())
    board = []
    for row in puzzle:
        for col in row:
            board.append(col)
    return board  #  el tablero es un vector con las filas concatenadas
  
#readBoardFromFile('/content/puzzleA.txt')

In [ ]:
def init_population(pop_number, size_chromosoma):
    population = []
    state_length=2
    ## Crea la poblacion inicial con cromosomas aleatorios (permutaciones)
    for i in range(pop_number):
        new_chromosome = [randint(0,9) for j in range(0,size_chromosoma)]
        #shuffle(new_chromosome)
        population.append(Individual(new_chromosome))
    return population

Estas son funciones utilitarias para trabajar con tableros de sudoku

In [ ]:
# devuelve los indices de los elementos de la columna i del tablero  
def get_column_indices(i):  
    indices = [i + 9 * j for j in range(9)]
    return indices

# devuelve los indices de los elementos de la fila i del tablero 
def get_row_indices(i):  
    indices = [j + 9*i for j in range(9)]
    return indices

# devuelve los indices de los elementos del bloque k del tablero. initialEntries son los indices de las posiciones inmutables (numeros iniciales) 
# si ignore_originals=true  entonces solo devuelve los indices de las posiciones que no son las posiciones inmutables
def get_block_indices(k, initialEntries, ignore_originals=False): 
    row_offset = (k//3)*3
    col_offset= (k%3)*3
    indices=[col_offset+(j%3)+9*(row_offset+(j//3)) for j in range(9)]
    if ignore_originals:
        indices = [x for x in indices if x not in initialEntries]
    return indices

# Completa aleatoriamente las pociciones vacias (que no son initialEntries). 
# Garantiza que en cada bloque se generen 9 numeros diferentes 
def randomAssign(board, initialEntries):
    for num in range(9):
        block_indices = get_block_indices(num, initialEntries)
        block = board[block_indices]
        zero_indices=[ind for i,ind in enumerate(block_indices) if block[i] == 0]
        to_fill = [i for i in range(1,10) if i not in block]
        shuffle(to_fill)
        for ind, value in zip(zero_indices, to_fill):
            board[ind] = value
    return board

def showBoard(board):
    board=np.array(board)
    def checkZero(s):
        if s != 0: return str(s)
        if s == 0: return "0"
    results = np.array([board[get_row_indices(j)] for j in range(9)])
    s=""
    for i, row in enumerate(results):
        if i%3==0:
            s +="-"*25+'\n'
        s += "| " + " | ".join([" ".join(checkZero(s) for s in list(row)[3*(k-1):3*k]) for k in range(1,4)]) + " |\n"
    s +="-"*25+''
    print (s)

Funcion para mostrar una población y su fitnes 

In [ ]:
def display_population(population):
    listaAG=[]
    for i in range(len(population)):
        listaAG.append([population[i].chromosome,population[i].fitness])

    data=pd.DataFrame(listaAG)
    data.columns = ['Poblacion','fitness']
    return data

### Funcion de Fitness

Funcion para evaluar el fitness de un Cromosoma (tablero de n-reinas):  nro de pares de reinas no atacadas (maximo valor = n*(n-1)/2) 

In [ ]:
# Evalua un tablero. Devuelve la cantidad de numeros diferentes en todas las filas y columnas 
# un tablero solucion tiene un score de 162 (81+81)
def fitness_soduko(chromosome):
    chromosome=np.array(chromosome)
    """Retorna el fitness de un cromosoma en el problema"""
    fitness = 0
    for row in range(9): # por cada fila obtiene la cantidad de numeros diferentes
        fitness+= len(set(chromosome[get_row_indices(row)]))
    for col in range(9): # por cada columna obtiene la cantidad de numeros diferentes
        fitness += len(set(chromosome[get_column_indices(col)]))
    return fitness

Funcion para evaluar toda una población de individuos con la funcion de fitnes especificada

In [ ]:
def evaluate_population(population, fitness_soduko):
    """ Evalua una poblacion de individuos con la funcion de fitness pasada """
    for i in range(len(population)):
        if population[i].fitness == -1:    # si el individuo no esta evaluado
            population[i].fitness = fitness_soduko(population[i].chromosome)
    return population

### Operadores de seleccion de padres

#### Seleccion por Ruleta

In [ ]:
def select_parents_roulette(population):
    popsize = len(population)
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
     
    # Escoje el segundo padre, desconsiderando el primer padre
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

#### Seleccion por Torneo

In [ ]:
def select_parents_torneo(population,size_torneo):
    
    # Escoje el primer padre
    list_indiv=[]
    x1 = np.random.permutation(len(population) )
    y1= x1[0:size_torneo]
    for i in range(size_torneo):
        list_indiv.append(population[y1[i]].fitness)
    
    iParent1=np.argmax(list_indiv)
    
    # Escoje el segundo padre, desconsiderando el primer padre   
    x2 = np.delete(x1, iParent1)
    x2 = np.random.permutation(x2)
    list_indiv=[]
    y2= x2[0:size_torneo]
    for i in range(size_torneo):
        list_indiv.append(population[y2[i]].fitness)
    iParent2=np.argmax(list_indiv)
    
    return (population[x1[iParent1]],population[x2[iParent2]])

## Seleccion de sobrevivientes por Ranking

#### Seleccion de sobrevivientes por ranking

In [ ]:
def select_survivors_ranking(population, offspring_population, numsurvivors):
    next_population = []
    population.extend(offspring_population) # une las dos poblaciones
    isurvivors = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:numsurvivors]
    for i in range(numsurvivors): next_population.append(population[isurvivors[i]])
    return next_population

## Algoritmo Genetico
Recibe una poblacion inicial, funcion de fitness, numero de generaciones (ngen), metodoSeleccion (ruleta/torneo), tournament_size, taza de mutación (pmut), operador de cruzamiento (crossover) y operador de mutacion (mutation)

In [ ]:
def genetic_algorithm(poblacion_inicial, fitness_fn, ngen=100, pmut=0.1, tournament_size=3,
                      crossover="onepoint", mutation="flip", 
                      selection_parents_method="roulette", 
                      selection_survivors_method="ranking"):
    """Algoritmo Genetico para el problema de la mochila
        ngen:       maximo numero de generaciones 
        pmut:       tasa de mutacion
        crossover:  operador de cruzamiento
        mutation:   operador de mutacion
        selection_parents_method: método de selección de padres para cruzamiento
        selection_survivors_method: método de selección de sobrevivientes 
    """
    population = deepcopy(poblacion_inicial)  # copia la poblacion inicial en una nueva variable para no alterar la inicial
    
    popsize = len(population)
    
    # Evalua la poblacion inicial
    evaluate_population(population, fitness_fn)  
    
    ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
    bestfitness = [population[ibest[0]].fitness]
    print("Poblacion inicial, best_fitness = {}".format(population[ibest[0]].fitness))
    
    # reemplaza los valores Nulos o 0 de los cromosomas por un valor del 1 al 9
    for i in range(len(population)):
      crom=np.array(population[i].chromosome)
      initialEntries = np.arange(81)[crom > 0]
      population[i].chromosome=randomAssign(crom, initialEntries).tolist()

    for g in range(ngen):   # Por cada generacion
        
        ## Selecciona las parejas de padres (mating pool) para cruzamiento 
        mating_pool = []
        if selection_parents_method=="roulette":
            for i in range(int(popsize/2)): mating_pool.append(select_parents_roulette(population)) 
        elif selection_parents_method=="tournament" :
            for i in range(int(popsize/2)): mating_pool.append(select_parents_torneo(population,tournament_size))
        else:
            raise NotImplementedError
                
        ## Crea la poblacion descendencia cruzando las parejas del mating pool 
        offspring_population = []
        for i in range(len(mating_pool)): 
            if crossover == "onepoint":# cruzamiento 1 punto
                offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) ) 
            elif crossover == "uniform":# cruzamiento uniforme
                offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) ) 
            elif crossover == "orderpermutation":# cruzamiento uniforme
                offspring_population.extend( mating_pool[i][0].crossover_orderpermutation(mating_pool[i][1]) ) 
            else:
                raise NotImplementedError                
           
        ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
        for i in range(len(offspring_population)):
            if uniform(0, 1) < pmut: 
                if mutation == "position":
                    offspring_population[i] = offspring_population[i].mutate_position()   # mutacion de una posicion
                elif mutation == "swap":
                    offspring_population[i] = offspring_population[i].mutate_swap()      # mutacion swap
                elif mutation == "neighborBoard":
                    offspring_population[i] = offspring_population[i].mutate_make_neighborBoard()      # mutacion make neighborBoard
                else:
                    raise NotImplementedError
        
        ## Evalua la poblacion descendencia creada
        evaluate_population(offspring_population, fitness_fn)  
        
        ## Selecciona popsize individuos para la sgte. generación 
        if selection_survivors_method == "ranking":
            population = select_survivors_ranking(population, offspring_population, popsize) #metodo de ranking
        else:
            raise NotImplementedError        
  
        ## Almacena la historia del fitness del mejor individuo
        ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
        bestfitness.append(population[ibest[0]].fitness)
        print("generacion {}, best_fitness = {},best_cromosoma = {}".format(g, population[ibest[0]].fitness,population[ibest[0]].chromosome))
    
    return population[ibest[0]], bestfitness,population[ibest[0]].chromosome 

## Probando el Algoritmo genetico

In [ ]:
## Crea la poblacion inicial con cromosomas aleatorios
size_chromosoma = 81    
num_individuals = 10

# Inicializa una poblacion inicial de forma aleatoria
poblacion_inicial = init_population(num_individuals,size_chromosoma)

Llama al algoritmo genetico para encontrar una solucion al problema de las n reinas

In [ ]:
import matplotlib.pyplot as plt
ngen = 1000 # Nro de generaciones
pmut = 1 # taza de mutación
fitness_fn=fitness_soduko

metodoSeleccion=['roulette','tournament']
tournament_size = 3 #tamaño del torneo = 3

crossover=['onepoint','uniform']
mutation=['position','swap','neighborBoard']

lst_bestcrom=[]
best_ind, bestfitness,best_chromosoma = genetic_algorithm(poblacion_inicial, fitness_fn, ngen, pmut, tournament_size, 
                                          crossover[1], mutation[1], metodoSeleccion[1])
lst_bestcrom.append(best_ind)
plt.plot(bestfitness)
plt.xlabel('Generacion')
plt.ylabel('Fitness')
plt.show()
print("* Merjor Resultado *")
print("Generacion : {} , max de fitness: {} ,  Cromosoma: {} \n".format(bestfitness.index(max(bestfitness)),np.max(bestfitness),best_chromosoma))

In [ ]:
# Grafica del tablero de soduko con la mejor solucion
showBoard(best_chromosoma)

-------------------------
| 3 5 8 | 4 6 2 | 7 9 1 |
| 4 1 5 | 3 2 8 | 6 7 9 |
| 7 8 3 | 5 1 9 | 4 6 5 |
-------------------------
| 1 7 9 | 6 5 3 | 2 8 4 |
| 8 4 7 | 2 9 5 | 1 3 6 |
| 6 3 4 | 9 7 1 | 8 5 2 |
-------------------------
| 5 1 2 | 8 3 6 | 9 4 8 |
| 8 9 6 | 1 4 7 | 5 2 3 |
| 9 6 1 | 5 8 4 | 3 1 7 |
-------------------------


In [ ]:
import matplotlib.pyplot as plt
## Crea la poblacion inicial con cromosomas aleatorios
size_chromosoma = 81    
num_individuals = 10

# Inicializa una poblacion inicial de forma aleatoria
poblacion_inicial = init_population(num_individuals,size_chromosoma)

ngen = 1000 # Nro de generaciones
pmut = {0.00,0.25,0.50,0.75,1.00} # taza de mutación
fitness_fn=fitness_soduko

metodoSeleccion=['roulette','tournament']
tournament_size = 3 #tamaño del torneo = 3

crossover=['onepoint','uniform']
mutation=['position','swap','neighborBoard']

Table_Resumen=[]
for m in metodoSeleccion:
  for j in crossover:
    for i in mutation:
      for l in pmut:
        lst_bestcrom = []
        print("******** Met_Seleccion : {}   ,  crossover : {}   ,  mutation : {}   ,    pmut : {}********".format(m,j,i,l))
        for a in range(10):
          print("\nIteracion {}".format(a+1))
          lst_bestcrom=[]
          best_ind, bestfitness,best_chromosoma = genetic_algorithm(poblacion_inicial, fitness_fn, ngen, l, tournament_size,
                                                    j, i,m)
          lst_bestcrom.append(best_ind)
          plt.plot(bestfitness)
          plt.show()
          print("* Merjor Resultado *")
          print("Generacion : {} , max de fitness: {} ,  Cromosoma: {} \n".format(bestfitness.index(max(bestfitness)),np.max(bestfitness),best_chromosoma))
          Table_Resumen.append((m,j,i,l,a, bestfitness.index(max(bestfitness)),np.max(bestfitness),best_chromosoma))

In [ ]:
import pandas as pd

resultados =pd.DataFrame(Table_Resumen)
resultados.columns=['Metodo Seleccion','Cruzamiento','Mutacion','Pm','Corrida','Generacion_Convergencia','Maximo_fitness','Cromosoma']
resultados

,Metodo Seleccion,Cruzamiento,Mutacion,Pm,Corrida,Generacion_Convergencia,Maximo_fitness,Cromosoma
0,roulette,onepoint,position,0.00,0,0,122,"[2, 8, 1, 1, 7, 4, 6, 4, 4, 5, 6, 8, 6, 8, 9, ..."
1,roulette,onepoint,position,0.00,1,0,122,"[2, 8, 1, 1, 7, 4, 8, 4, 4, 4, 6, 8, 6, 8, 2, ..."
2,roulette,onepoint,position,0.00,2,48,123,"[2, 4, 1, 6, 8, 8, 7, 3, 9, 7, 3, 3, 9, 3, 2, ..."
3,roulette,onepoint,position,0.00,3,0,122,"[2, 8, 1, 1, 7, 4, 9, 4, 4, 7, 6, 8, 6, 8, 2, ..."
4,roulette,onepoint,position,0.00,4,0,122,"[2, 8, 1, 1, 7, 4, 8, 4, 4, 9, 6, 8, 6, 8, 9, ..."
5,roulette,onepoint,position,0.00,5,34,126,"[2, 8, 1, 1, 7, 4, 9, 4, 4, 4, 6, 8, 6, 8, 2, ..."
6,roulette,onepoint,position,0.00,6,0,122,"[2, 8, 1, 1, 7, 4, 6, 4, 4, 4, 6, 8, 6, 8, 9, ..."
7,roulette,onepoint,position,0.00,7,0,122,"[2, 8, 1, 1, 7, 4, 2, 4, 4, 9, 6, 8, 6, 8, 3, ..."
8,roulette,onepoint,position,0.00,8,0,122,"[2, 8, 1, 1, 7, 4, 2, 4, 4, 5, 6, 8, 6, 8, 9, ..."
9,roulette,onepoint,position,0.00,9,0,122,"[2, 8, 1, 1, 7, 4, 9, 4, 4, 5, 6, 8, 6, 8, 2, ..."


In [ ]:
resultados.iloc[resultados["Maximo_fitness"].idxmax()]

Metodo Seleccion                                                    roulette
Cruzamiento                                                         onepoint
Mutacion                                                                swap
Pm                                                                       1.0
Corrida                                                                    0
Generacion_Convergencia                                                  800
Maximo_fitness                                                           160
Cromosoma                  [2, 5, 6, 7, 3, 9, 8, 4, 1, 4, 7, 2, 6, 8, 9, ...
Name: 80, dtype: object

In [ ]:
# Se busca el cromosoma que tubo el maximo fitnes
mejor_cromosoma=resultados.iloc[resultados["Maximo_fitness"].idxmax()]['Cromosoma']
# Grafica del tablero de soduko con la mejor solucion
showBoard(mejor_cromosoma)

-------------------------
| 2 5 6 | 7 3 9 | 8 4 1 |
| 4 7 2 | 6 8 9 | 3 1 5 |
| 6 9 4 | 8 5 1 | 7 3 2 |
-------------------------
| 5 4 7 | 2 9 3 | 1 8 6 |
| 8 3 1 | 9 6 4 | 2 5 7 |
| 7 8 5 | 3 1 2 | 4 9 9 |
-------------------------
| 9 1 8 | 4 2 7 | 5 6 3 |
| 1 6 3 | 5 7 8 | 9 2 4 |
| 3 2 9 | 1 4 5 | 6 7 8 |
-------------------------


In [ ]:
tabla_resumen=pd.DataFrame(resultados.groupby(['Metodo Seleccion','Cruzamiento','Mutacion','Pm'])[['Generacion_Convergencia','Maximo_fitness']].mean())
tabla_resumen.to_csv('tabla_resumen_fitness.csv')
tabla_resumen

Generacion_Convergencia  \
Metodo Seleccion Cruzamiento Mutacion      Pm                              
roulette         onepoint    neighborBoard 0.00                      8.0   
                                           0.25                     15.6   
                                           0.50                      3.6   
                                           0.75                     26.7   
                                           1.00                      4.8   
                             position      0.00                      8.2   
                                           0.25                    543.9   
                                           0.50                    511.2   
                                           0.75                    326.4   
                                           1.00                    422.3   
                             swap          0.00                     21.2   
                                           0.25                    752.2   
                                           0.50                    579.1   
                                           0.75                    660.6   
                                           1.00                    654.1   
                 uniform     neighborBoard 0.00                     25.1   
                                           0.25                     25.7   
                                           0.50                     30.3   
                                           0.75                     22.1   
                                           1.00                     23.1   
                             position      0.00                    105.8   
                                           0.25                    657.6   
                                           0.50                    384.2   
                                           0.75                    286.7   
                                           1.00                    564.4   
                             swap          0.00                     21.9   
                                           0.25                    794.2   
                                           0.50                    788.3   
                                           0.75                    711.7   
                                           1.00                    658.9   
tournament       onepoint    neighborBoard 0.00                      0.0   
                                           0.25                      0.0   
                                           0.50                     10.3   
                                           0.75                      0.7   
                                           1.00                     20.5   
                             position      0.00                      5.4   
                                           0.25                    602.8   
                                           0.50                    396.3   
                                           0.75                    278.3   
                                           1.00                    408.3   
                             swap          0.00                      2.4   
                                           0.25                    724.3   
                                           0.50                    774.7   
                                           0.75                    707.9   
                                           1.00                    604.2   
                 uniform     neighborBoard 0.00                     11.4   
                                           0.25                     10.9   
                                           0.50                     24.7   
                                           0.75                     36.0   
                                           1.00                     11.1   
                             position      0.00                     22.2   
                     

In [ ]:
def init_population_ruta(lista):
    population = []
    for i in lista:
        new_chromosome = readBoardFromFile(i)
        population.append(Individual(new_chromosome))
    return population

In [ ]:
# Inicializa una poblacion inicial de una ruta o lista de archivos
lista=['/content/puzzleA.txt','/content/puzzleB.txt','/content/puzzleC.txt','/content/puzzleD.txt','/content/puzzleE.txt']
poblacion_inicial = init_population_ruta(lista)

In [ ]:
import matplotlib.pyplot as plt
ngen = 10000 # Nro de generaciones
pmut = 1 # taza de mutación
fitness_fn=fitness_soduko

metodoSeleccion=['roulette','tournament']
tournament_size = 3 #tamaño del torneo = 3

crossover=['onepoint','uniform']
mutation=['position','swap','neighborBoard']

lst_bestcrom=[]
best_ind, bestfitness,best_chromosoma = genetic_algorithm(poblacion_inicial, fitness_fn, ngen, pmut, tournament_size, 
                                          crossover[1], mutation[1], metodoSeleccion[0])
lst_bestcrom.append(best_ind)
plt.plot(bestfitness)
plt.show()
print("* Merjor Resultado *")
print("Generacion : {} , max de fitness: {} ,  Cromosoma: {} \n".format(bestfitness.index(max(bestfitness)),np.max(bestfitness),best_chromosoma))

In [ ]:
# Grafica del tablero de soduko con la mejor solucion
showBoard(best_chromosoma)

-------------------------
| 6 3 1 | 4 1 5 | 2 8 7 |
| 2 4 9 | 1 3 7 | 5 6 8 |
| 1 5 3 | 7 8 4 | 9 2 6 |
-------------------------
| 5 9 5 | 3 6 8 | 1 4 2 |
| 4 6 7 | 8 9 2 | 3 1 5 |
| 7 8 2 | 1 4 5 | 6 9 3 |
-------------------------
| 3 2 6 | 5 4 1 | 8 7 9 |
| 8 1 4 | 9 5 6 | 7 3 4 |
| 9 7 8 | 6 2 3 | 4 5 1 |
-------------------------
